In [1]:
import pandas as pd

In [2]:
imdb_titles_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.basics.tsv'
imdb_ratings_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.ratings.tsv'
imdb_principals_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.principals.tsv'
imdb_crew_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.crew.tsv'
imdb_names_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/name.basics.tsv'
rotten_tomatoes_movies_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/rotten_tomatoes/rotten_tomatoes_movies.csv'
netflix_titles_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/netflix/netflix_titles.csv'

## IMDB dataset

In [20]:
imdb_titles_df = pd.read_csv(imdb_titles_path, '\t', low_memory=False)
imdb_titles_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


In [21]:
imdb_ratings_df = pd.read_csv(imdb_ratings_path,'\t')
imdb_ratings_df.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1720
1,tt0000002,6.0,211


In [22]:
imdb_principals_df = pd.read_csv(imdb_principals_path,'\t')
imdb_principals_df.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N


In [23]:
imdb_names_df = pd.read_csv(imdb_names_path,'\t')
imdb_names_df.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0050419,tt0031983,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0071877,tt0038355"


## Rotten Tomatoes Dataset

In [24]:
rt_df = pd.read_csv(rotten_tomatoes_movies_path)
rt_df.head(2)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19


## Netflix Dataset

In [28]:
netflix_df = pd.read_csv(netflix_titles_path)
netflix_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...


## Tables Create Statements

In [3]:
import psycopg2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
def create_database():

    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=antoniam")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("""
    SELECT pg_terminate_backend(pg_stat_activity.pid)
    FROM pg_stat_activity
    WHERE pg_stat_activity.datname = 'movies_db';""")
    cur.execute("DROP DATABASE IF EXISTS movies_db")
    cur.execute("CREATE DATABASE movies_db WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=movies_db")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

In [5]:
cur, conn = create_database()

In [6]:
create_staging_imdb_titles ="""CREATE TABLE IF NOT EXISTS staging_imdb_titles(
tconst VARCHAR NOT NULL PRIMARY KEY,
titleType VARCHAR,
primaryTitle VARCHAR,
originalTitle VARCHAR,
isAdult VARCHAR,
startYear VARCHAR,
endYear VARCHAR,
runtimeMinutes VARCHAR,
genres VARCHAR);
"""

create_staging_imdb_ratings ="""CREATE TABLE IF NOT EXISTS staging_imdb_ratings(
tconst VARCHAR NOT NULL,
averageRating FLOAT,
numVotes INT);
"""

create_staging_imdb_principals = """CREATE TABLE IF NOT EXISTS staging_imdb_principals(
tconst VARCHAR NOT NULL,
ordering INTEGER NOT NULL,
nconst VARCHAR NOT NULL, 
category VARCHAR NOT NULL,
job VARCHAR NOT NULL,
characters VARCHAR NOT NULL);
"""

create_staging_imdb_crew = """CREATE TABLE IF NOT EXISTS staging_imdb_crew(
tconst VARCHAR NOT NULL,
directors VARCHAR, 
writers VARCHAR);
"""

create_staging_imdb_names = """CREATE TABLE IF NOT EXISTS staging_imdb_names(
nconst VARCHAR NOT NULL PRIMARY KEY,
primaryName VARCHAR NOT NULL,
birthYear VARCHAR,
deathYear VARCHAR,
primaryProfession VARCHAR,
knownForTitles VARCHAR);
"""

create_staging_rotten_tomatoes_titles = """CREATE TABLE IF NOT EXISTS staging_rotten_tomatoes_titles(
rotten_tomatoes_link VARCHAR, 
movie_title VARCHAR,
movie_info VARCHAR,
critics_consensus VARCHAR, 
content_rating VARCHAR, 
genres VARCHAR, 
directors VARCHAR, 
authors VARCHAR,
actors VARCHAR, 
original_release_date DATE, 
streaming_release_date DATE, 
runtime FLOAT,
production_company VARCHAR, 
tomatometer_status VARCHAR, 
tomatometer_rating FLOAT,
tomatometer_count INT, 
audience_status VARCHAR, 
audience_rating FLOAT,
audience_count INT, 
tomatometer_top_critics_count INT,
tomatometer_fresh_critics_count INT, 
tomatometer_rotten_critics_count INT);
"""
create_staging_netflix_titles = """CREATE TABLE IF NOT EXISTS staging_netflix_titles(
show_id VARCHAR NOT NULL PRIMARY KEY,
"type" VARCHAR,
title VARCHAR,
director VARCHAR,
"cast" VARCHAR,
country VARCHAR,
date_added DATE,
release_year INT4,
rating VARCHAR,
duration VARCHAR,
listed_in VARCHAR,
description VARCHAR);
"""

In [10]:
create_temp_table = \
"""CREATE TABLE IF NOT EXISTS temp_netflix_imdb(
    imdb_title_id VARCHAR,
    netflix_title_id VARCHAR);
"""

In [11]:
create_titles = \
"""CREATE TABLE IF NOT EXISTS titles(
  id SERIAL PRIMARY KEY,
  name VARCHAR,
  description VARCHAR,
  year INT4,
  runtime_minutes INT,
  country VARCHAR,
  isAdult BOOLEAN,
  type VARCHAR,
  imdb_avg_score FLOAT,
  imdb_n_ratings INT,
  rt_critics_score FLOAT,
  rt_n_critics INT,
  rt_audience_score FLOAT,
  rt_n_audience INT);
"""

create_roles = \
"""CREATE TABLE IF NOT EXISTS roles(
id SERIAL PRIMARY KEY,
title_id VARCHAR,
person_id VARCHAR,
role_name VARCHAR NOT NULL
);
"""

create_persons = \
"""CREATE TABLE IF NOT EXISTS persons(
id VARCHAR NOT NULL PRIMARY KEY,
full_name VARCHAR NOT NULL,
birth_year INT4,
death_year INT4
);
"""

create_genres = \
"""CREATE TABLE IF NOT EXISTS genres(
id SERIAL PRIMARY KEY,
title_id VARCHAR NOT NULL,
genre_name VARCHAR NOT NULL);"""


In [12]:
staging_tables_sql = [create_staging_imdb_titles, create_staging_imdb_ratings, create_staging_imdb_principals, create_staging_imdb_names, 
 create_staging_rotten_tomatoes_titles, create_staging_netflix_titles, create_staging_imdb_crew, create_temp_table]

final_tables = [create_titles, create_roles, create_persons, create_genres]

In [13]:
for s in staging_tables_sql + final_tables:
    cur.execute(s)

In [14]:
copy_staging_titles = f"COPY staging_imdb_titles FROM '{imdb_titles_path}' delimiter '\t' CSV HEADER"
copy_staging_ratings = f"COPY staging_imdb_ratings FROM '{imdb_ratings_path}' delimiter '\t' CSV HEADER"
copy_staging_principals = f"COPY staging_imdb_principals FROM '{imdb_principals_path}' delimiter '\t' CSV HEADER"
copy_staging_crew = f"COPY staging_imdb_crew FROM '{imdb_crew_path}' delimiter '\t' CSV HEADER"
copy_staging_names = f"COPY staging_imdb_names FROM '{imdb_names_path}' delimiter '\t' CSV HEADER"
copy_staging_rt = f"COPY staging_rotten_tomatoes_titles FROM '{rotten_tomatoes_movies_path}' delimiter ',' CSV HEADER"
copy_staging_netflix = f"COPY staging_netflix_titles FROM '{netflix_titles_path}' delimiter ',' CSV HEADER"


copy_staging = [copy_staging_titles, copy_staging_ratings, copy_staging_principals, copy_staging_names, copy_staging_crew, copy_staging_rt, copy_staging_netflix]

for s in copy_staging: 
    print(s)
    cur.execute(s)

COPY staging_imdb_titles FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.basics.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_ratings FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.ratings.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_principals FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.principals.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_names FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/name.basics.tsv' delimiter '	' CSV HEADER
COPY staging_imdb_crew FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.crew.tsv' delimiter '	' CSV HEADER
COPY staging_rotten_tomatoes_titles FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/rotten_tomatoes/rotten_tomatoes_movies.csv' delimiter ',' CSV HEADER
COPY staging_netflix_titles FROM '/Users/antoniam/Desktop/personal/netflix-ratings/data/netflix/netflix_titles.csv' delimiter ',' CSV HEADER


## Final tables

### Temp table

In [41]:
cur.execute('DROP table temp_netflix_imdb')

In [43]:
cur.execute(create_temp_table)

In [44]:
from time import time

In [45]:
t = time()
st = \
"""INSERT INTO temp_netflix_imdb(imdb_title_id, netflix_title_id)
SELECT imdb.tconst, netflix.show_id
FROM staging_netflix_titles netflix 
LEFT JOIN staging_imdb_titles imdb
ON lower(substr(netflix.title, 1, 15)) = lower(substr(imdb.primaryTitle,1, 15))
"""
cur.execute(st)
print(time() - t)

112.63449692726135


In [47]:
cur.execute('SELECT * from temp_netflix_imdb limit(10)')
cur.fetchone()

('tt10620868', 's28')

### Genres table

In [60]:
cur.execute(create_genres)

In [61]:
st = \
"""INSERT INTO genres(title_id, genre_name)
   SELECT tconst, unnest(string_to_array(genres, ',')) as genre
   FROM temp_netflix_imdb
   LEFT JOIN staging_imdb_titles
   ON temp_netflix_imdb.imdb_title_id = staging_imdb_titles.tconst
   WHERE genres!='\\N'
   """
cur.execute(st)

### Persons table

In [66]:
st = \
"""INSERT INTO persons(id, full_name, birth_year, death_year)
SELECT nconst, primaryName, 
NULLIF(regexp_replace(birthYear, '\D','','g'), '')::numeric AS birth_year,
NULLIF(regexp_replace(deathYear, '\D','','g'), '')::numeric AS death_year
from staging_imdb_names
"""
cur.execute(st)

### Roles table

In [62]:
st = \
"""INSERT INTO roles(title_id, person_id, role_name)
SELECT tconst, nconst, category
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_principals 
ON temp_netflix_imdb.imdb_title_id = staging_imdb_principals.tconst
WHERE category in ('director', 'writer', 'actor', 'actress')

UNION
SELECT tconst, unnest(string_to_array(directors, ',')) as nconst, 'director'
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_crew
ON temp_netflix_imdb.imdb_title_id = staging_imdb_crew.tconst
WHERE directors!='\\N'

UNION
SELECT tconst, unnest(string_to_array(writers, ',')) as nconst, 'writer'
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_crew
ON temp_netflix_imdb.imdb_title_id = staging_imdb_crew.tconst
WHERE writers!='\\N'
"""
cur.execute(st)

### Titles table

In [67]:
netflix_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [68]:
imdb_titles_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8078842,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8078843,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8078844,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8078845,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [106]:
st = """SELECT *
FROM temp_netflix_imdb
JOIN staging_netflix_titles netflix
ON temp_netflix_imdb.netflix_title_id = netflix.show_id
JOIN staging_imdb_titles imdb
ON temp_netflix_imdb.imdb_title_id = imdb.tconst
JOIN (
    SELECT staging_netflix_titles.show_id, unnest(string_to_array(staging_netflix_titles.director, ',')) 
    FROM staging_netflix_titles) as director
ON director.show_id = netflix.show_id
WHERE netflix.release_year::varchar = imdb.startYear
 
"""
cur.execute(st)

In [107]:
cur.fetchone()

('tt0034498',
 's6118',
 's6118',
 'Movie',
 'The Battle of Midway',
 'John Ford',
 'Henry Fonda, Jane Darwell',
 'United States',
 datetime.date(2017, 3, 31),
 1942,
 'TV-14',
 '18 min',
 'Classic Movies, Documentaries',
 'Director John Ford captures combat footage of the Battle of Midway, an air and sea campaign that was a turning point in the war in the Pacific.',
 'tt0034498',
 'short',
 'The Battle of Midway',
 'The Battle of Midway',
 '0',
 '1942',
 '\\N',
 '18',
 'Documentary,Short,War',
 's6118',
 'John Ford')

In [ ]:
(SELECT id, unnest(string_to_array(news_article.news_category_id, ',')):: int rowz
      FROM news_article) AS c ON c.id = a.id

In [101]:
cur.execute("SELECT staging_netflix_titles.show_id, unnest(string_to_array(staging_netflix_titles.director, ',')) from staging_netflix_titles")

In [75]:
st = """SELECT COUNT(*) 
FROM temp_netflix_imdb
JOIN staging_netflix_titles netflix
ON temp_netflix_imdb.netflix_title_id = netflix.show_id
JOIN staging_imdb_titles imdb
ON temp_netflix_imdb.imdb_title_id = imdb.tconst
WHERE netflix.release_year::varchar = imdb.startYear"""
cur.execute(st)

In [80]:
cur.fetchone()

('William Wyler',)

In [49]:
cur.execute("SELECT levenshtein('GUMBO', 'GAMBOL')")
cur.fetchone()

(2,)